In [ ]:
#to load dataset
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Read the CSV file
df = pd.read_csv('/content/top250-00-19.csv')

# Merge Bundesliga leagues and LaLiga leagues to avoid duplicates
df['League_from'] = df['League_from'].replace({'1.Bundesliga': 'Bundesliga', '2.Bundesliga': 'Bundesliga', 'LaLiga2': 'LaLiga'})
df['League_to'] = df['League_to'].replace({'1.Bundesliga': 'Bundesliga', '2.Bundesliga': 'Bundesliga', 'LaLiga2': 'LaLiga'})

# Figuring out top 5 leagues
top_leagues = ['Bundesliga', 'LaLiga', 'League One', 'Premier League', 'Serie A']
df_top = df[df['League_from'].isin(top_leagues) | df['League_to'].isin(top_leagues)]

# Convert Transfer_fee to numeric and clearing all other non-numeric
df_top['Transfer_fee'] = pd.to_numeric(df_top['Transfer_fee'], errors='coerce')

#distribution of transfer fees
plt.figure(figsize=(10, 6))
plt.hist(df_top['Transfer_fee'].dropna(), bins=30, edgecolor='black')
plt.title('Distribution of Transfer Fees')
plt.xlabel('Transfer Fee')
plt.ylabel('Frequency')
plt.show()

# Calculating Average transfer fees by season
avg_fees = df_top.groupby('Season')['Transfer_fee'].mean().reset_index()

plt.figure(figsize=(10, 6))
plt.plot(avg_fees['Season'], avg_fees['Transfer_fee'], marker='o')
plt.title('Average Transfer Fee by Season')
plt.xlabel('Season')
plt.ylabel('Average Transfer Fee')
plt.xticks(rotation=45)
plt.show()

# Checking the impact of Player's Age in Transfer fees
plt.figure(figsize=(10, 6))
plt.scatter(df_top['Age'], df_top['Transfer_fee'])
plt.title('Age vs Transfer Fee')
plt.xlabel('Age')
plt.ylabel('Transfer Fee')
plt.show()

# Checking the impact of Player's Position in Transfer fees
plt.figure(figsize=(12, 6))
sns.boxplot(x='Position', y='Transfer_fee', data=df_top)
plt.title('Transfer Fee Distribution by Position')
plt.xlabel('Position')
plt.ylabel('Transfer Fee')
plt.xticks(rotation=90)
plt.show()

# Checking the Transfer fee distribution by League (From)
plt.figure(figsize=(10, 6))
sns.boxplot(x='League_from', y='Transfer_fee', data=df_top[df_top['League_from'].isin(top_leagues)])
plt.title('Transfer Fee Distribution by League (From)')
plt.xlabel('League')
plt.ylabel('Transfer Fee')
plt.xticks(rotation=45)
plt.show()

# Figuring out top 10 High-profile Transfers
top_10_transfers = df_top.nlargest(10, 'Transfer_fee')[['Name', 'Position', 'Team_from', 'Team_to', 'Season', 'Transfer_fee']]
print(top_10_transfers.to_string(index=False))

# Calculatingb the Transfer Premium Analysis
df_top['Fee_per_year'] = df_top['Transfer_fee'] / df_top['Age']
top_10_premiums = df_top.nlargest(10, 'Fee_per_year')[['Name', 'Age', 'Team_from', 'Team_to', 'Season', 'Transfer_fee', 'Fee_per_year']]
print(top_10_premiums.to_string(index=False))

# Checking Club's Strategies of Buying Players at Low value
young_high_value = df_top[df_top['Age'] <= 23].nlargest(10, 'Transfer_fee')[['Name', 'Age', 'Team_from', 'Team_to', 'Season', 'Transfer_fee']]
print(young_high_value.to_string(index=False))

# Checking Club's Strategies of Buying Players at Low value and Selling High
high_value_transfers = df_top[df_top['Transfer_fee'] > df_top['Transfer_fee'].quantile(0.95)]
season_counts = high_value_transfers['Season'].value_counts().sort_index()

plt.figure(figsize=(12, 6))
season_counts.plot(kind='bar')
plt.title('High-Value Transfers by Season')
plt.xlabel('Season')
plt.ylabel('Number of High-Value Transfers')
plt.xticks(rotation=45)
plt.show()

# Figuring out top 10 value increase players
df_top['Value_increase'] = df_top['Transfer_fee'] - df_top['Market_value']
top_10_value_increase = df_top.nlargest(10, 'Value_increase')[['Name', 'Position', 'Team_from', 'Team_to', 'Season', 'Market_value', 'Transfer_fee', 'Value_increase']]
print("\nTop 10 Value Increases:")
print(top_10_value_increase.to_string(index=False))